## Q1 How many transactions in the dataset?

## Q2 What is the top-3 most expensive item in the database? 
* show the price of item.

## Q3 Show number of customers were born in each month.
* there are 12 months

## Q4 Show top-3 popular items that make most income.
* show the total income of each item.

## Q5 A manager want to crate a birth-month promotion in January 2017.
* How many customers were born in January and also made transcations in January 2017?

## Q6 In January, what were the top-3 favorite items that customer bought with "Bread"?
* show number of times

## Q7 Who are the top-3 customers that pay most money for "Tea"? 
* show total payment of each customer.

## Q8 What are the top-5 items that "Harry Brown" pay most money? 
* show total payment of each item.

## Q9 Who are top-5 customer paid most money in 2016? 
* show total payment of each customer

## Q10 Between January and March 2017, Who are top-3 customers that paid most for Coffee?
* show total payment

In [3]:
import pandas as pd
from django.db.models import Q

In [2]:
#Q1 How many transactions in the dataset?
Transaction.objects.aggregate(Count('id'))

{'id__count': 9531}

In [3]:
#Q2 What is the top-3 most expensive item in the database? 
# show the price of item.
x=Item.objects.order_by('-price')[:3]
pd.DataFrame(x)

,0
0,Jam 400.00
1,My-5 Fruit Shoot 400.00
2,Empanadas 400.00


In [4]:
#Q3 Show number of customers were born in each month.
# there are 12 months
x=Customer.objects.values('dob__month')\
    .annotate(Count('id'))
pd.DataFrame(x)

,dob__month,id__count
0,1,17
1,2,7
2,3,6
3,4,10
4,5,6
5,6,12
6,7,11
7,8,12
8,9,14
9,10,9


In [4]:
%%timeit -n1
#Q4 Show top-3 popular items that make most income.
# show the total income of each item.

x=Record.objects.values('item','item__name')\
    .annotate(Sum('item__price'))\
    .order_by('-item__price__sum')[:3]
pd.DataFrame(x)

9.5 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
#Q5 A manager want to crate a birth-month promotion in January 2017.
# How many customers were born in January and also made transcations in January 2017?

m=1
x=Transaction.objects.filter(datetime__month=m, datetime__year=2017)\
    .values(
        'datetime',
        'customer__firstname',
        'customer__lastname',
        'customer__dob'
    )\
    .filter(customer__dob__month=m)\
    .aggregate(Count('id'))
x
#pd.DataFrame(x)

{'id__count': 211}

In [11]:
#Q5 A manager want to crate a birth-month promotion in January 2017.
# How many customers were born in January and also made transcations in January 2017?

m=1
x=Transaction.objects.filter(
        datetime__month=m, 
        customer__dob__month=m,
        datetime__year=2017
    )\
    .values(
        'datetime',
        'customer__firstname',
        'customer__lastname',
        'customer__dob'
    )\
    .aggregate(Count('id'))

x

{'id__count': 211}

In [14]:
y=Transaction.objects.filter(
        datetime__month=m, 
        customer__dob__month=m,
        datetime__year=2017
    )\
    .values(
        'datetime',
        'customer__firstname',
        'customer__lastname',
        'customer__dob'
    )
print(y.query)

SELECT "myapp_transaction"."datetime", "myapp_customer"."firstname", "myapp_customer"."lastname", "myapp_customer"."dob" FROM "myapp_transaction" INNER JOIN "myapp_customer" ON ("myapp_transaction"."customer_id" = "myapp_customer"."id") WHERE (django_date_extract('month', "myapp_customer"."dob") = 1 AND django_datetime_extract('month', "myapp_transaction"."datetime", 'UTC') = 1 AND "myapp_transaction"."datetime" BETWEEN 2017-01-01 00:00:00 AND 2017-12-31 23:59:59.999999)


In [15]:
#Q6 In January, what were the top-3 favorite items that customer bought with "Bread"?
# show number of times
x=Record.objects.filter(
        transaction__record__item__name='Bread',
        transaction__datetime__month=1
    )\
    .values('item__name')\
    .annotate(Count('id'))\
    .order_by('-id__count')[1:4]
pd.DataFrame(x)

,id__count,item__name
0,172,Coffee
1,51,Tea
2,41,Cake


In [8]:
#Q7 Who are the top-3 customers that pay most money for "Tea"? 
# show total payment of each customer.
x=Record.objects.filter(item__name='Tea')\
    .values(
        'transaction__customer__id',
        'transaction__customer__firstname',
        'transaction__customer__lastname'
    )\
    .annotate(Sum('item__price'))\
    .order_by('-item__price__sum')[:3]
pd.DataFrame(x)

,item__price__sum,transaction__customer__firstname,transaction__customer__id,transaction__customer__lastname
0,7410,Jacob,54,Taylor
1,7020,Harry,77,Wilson
2,7020,Claire,84,Wilson


In [9]:
#Q8 What are the top-5 items that "Harry Brown" pay most money? 
# show total payment of each item.
x=Record.objects.filter(transaction__customer__firstname='Harry',transaction__customer__lastname='Brown')\
    .values('item__id','item__name')\
    .annotate(Sum('item__price'))\
    .order_by('-item__price__sum')[:5]
pd.DataFrame(x)

,item__id,item__name,item__price__sum
0,10,Coffee,16770
1,4,Bread,11560
2,13,Tea,6630
3,12,Medialuna,2880
4,6,Hot chocolate,2700


In [10]:
#Q9 Who are top-5 customer paid most money in 2016? 
# show total payment of each customer
x=Transaction.objects.filter(datetime__year=2016)\
    .values(
        'customer__firstname',
        'customer__lastname',
    )\
    .annotate(Sum('record__item__price'))\
    .order_by('-record__item__price__sum')[:5]
pd.DataFrame(x)

,customer__firstname,customer__lastname,record__item__price__sum
0,James,Johnson,38130
1,Harry,Wilson,33300
2,George,Smith,33260
3,Jack,Taylor,32540
4,Jack,Evans,32090


In [11]:
#Q10 Between January and March 2017, Who are top-3 customers that paid most for Coffee?
# show total payment

x=Transaction.objects.filter(
        datetime__month__gte=1,
        datetime__month__lte=3,
        datetime__year=2017,
        record__item__name='Coffee'
    )\
    .values(
        'customer__firstname',
        'customer__lastname',
    )\
    .annotate(Sum('record__item__price'))\
    .order_by('-record__item__price__sum')[:3]
pd.DataFrame(x)

,customer__firstname,customer__lastname,record__item__price__sum
0,Thomas,Johnson,16380
1,Bob,Johnson,15600
2,Claire,Wilson,14040
